In [ ]:
import tweepy
from config import settings
from pyairtable import Table
def divide_into_tweet(text):
    puncts = ['.', ',', ';', '--']
    tweets = []
    while len(text) > 280:
        cut_where, cut_why = max((text.rfind(punc, 0, 276), punc) for punc in puncts)
        if cut_where <= 0:
            cut_where = text.rfind(' ', 0, 276)
            cut_why = ' '
        cut_where += len(cut_why)
        tweets.append(text[:cut_where].rstrip())
        text = text[cut_where:].lstrip()
    tweets.append(text)        
    return tweets    

In [ ]:
# Authenticate to Twitter
auth = tweepy.OAuthHandler(settings.TWITTER_CONSUMER_KEY, settings.TWITTER_CONSUMER_SECRET)
auth.set_access_token(settings.TWITTER_ACCESS_TOKEN, settings.TWITTER_ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

try:
    api.verify_credentials()
    print("Authentication Successful")
except:
    print("Authentication Error")

In [ ]:
table = Table(api_key=settings.AIRTABLE_API_KEY, base_id=settings.AIRTABLE_BASE_ID, table_name="Tweets") 
for record in table.all():
    if record["fields"]["Posted"] == 0:
        break
    else:
        continue


In [ ]:
airtable_field_id = record['id']
author = record["fields"]['Author']
title = record["fields"]['Title']
quote = record["fields"]['Quote']
tweets = divide_into_tweet(quote)
reference = f"Taken from {title} by {author}"
first_tweet = None
counter = 0 
number_of_tweets = len(tweets)
if len(tweets) == 1:
    first_tweet = print(f"{tweets[0]}")
    print(f"{reference}")#, in_reply_to_status_id=first_tweet.id)
else:
    for tweet in tweets:
        counter = counter + 1
        if first_tweet is None:
            first_tweet = print(f"{tweet} ({counter}/{number_of_tweets})")
        else:
            print(f"{tweet} ({counter}/{number_of_tweets})")#, in_reply_to_status_id=first_tweet.id)
    print(f"{reference}")#, in_reply_to_status_id=first_tweet.id)


In [ ]:
table.batch_update(records=[{"id": airtable_field_id, "fields": {"Posted": True}}])